# Ternary free energy surface for IN625 from toy CALPHAD DB

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ternary
from pycalphad import Database, calculate, Model
from scipy.optimize import curve_fit
from scipy.interpolate import SmoothBivariateSpline
from IPython.display import display
#from sympy import init_printing
#init_printing()
from sympy import ccode

simplex_res = 50

In [2]:
#tdb = Database('mc_ni_v2.033.tdb')
tdb = Database('NiNbMo_FccDeltaMu_Thermotech.TDB')

In [3]:
phases = ['LIQUID', 'FCC_A1', 'MU_PHASE', 'DELTA']
elements = ['NI', 'MO', 'NB', 'VA']
temp = 800 + 273.15 # Kelvin

## Look into pycalphad.model

In [52]:
#tdb.phases['MU_PHASE']
constituents = list(set([i for c in tdb.phases['MU_PHASE'].constituents for i in c]))
m = Model(tdb, constituents, 'MU_PHASE')
#ccode(m.ast)
m.ast.xreplace?

In [60]:
from sympy.parsing.sympy_parser import parse_expr
from sympy import diff, symbols
x_mu_Ni, x_mu_Mo, x_mu_Nb = symbols('x_mu_Ni x_mu_Mo x_mu_Nb')
MU_PHASE2NI, MU_PHASE2MO, MU_PHASE2NB = symbols('MU_PHASE2NI MU_PHASE2MO MU_PHASE2NB')
expr = parse_expr(str(m.ast)) #.xreplace(MU_PHASE2NI: x_mu_Ni)#, MU_PHASE2MO: x_mu_Mo, MU_PHASE2NB: x_mu_Nb) #parse_expr(str(m.ast))
#print expr

dm = diff(expr, MU_PHASE2NI)
print dm

0.0769230769230769*MU_PHASE0NI*MU_PHASE1MO*Piecewise((-100*T + 9*Piecewise((-0.0048407*T**2 - 22.096*T*log(T) + 117.854*T - 5179.159, And(T < 1728.0, T >= 298.14)), (-43.1*T*log(T) + 279.135*T - 27840.655 + 1.12754e+31/T**9, And(T < 3000.0, T >= 1728.0)), (0, True)) + 4*Piecewise((-1.309265e-10*T**4 + 5.66283e-7*T**3 - 0.003443396*T**2 - 23.56414*T*log(T) + 131.9197*T - 7746.302 + 65812/T, And(T < 2896.0, T >= 298.14)), (-42.63829*T*log(T) + 283.559746*T - 30556.41 - 4.849315e+33/T**9, And(T < 4000.0, T >= 2896.0)), (0, True)) + 156150, And(T < 6000.0, T >= 298.15)), (0, True)) + 0.0769230769230769*MU_PHASE0NI*MU_PHASE1NB*Piecewise((50*T + 9*Piecewise((-0.0048407*T**2 - 22.096*T*log(T) + 117.854*T - 5179.159, And(T < 1728.0, T >= 298.14)), (-43.1*T*log(T) + 279.135*T - 27840.655 + 1.12754e+31/T**9, And(T < 3000.0, T >= 1728.0)), (0, True)) + 4*Piecewise((-3.50119e-7*T**3 + 0.000203475*T**2 - 26.4711*T*log(T) + 142.045475*T - 8519.353 + 93399/T, And(T < 2750.0, T >= 298.14)), (-41.77*T*

## Visualize free energy of γ

In [ ]:
gamma_data = calculate(tdb, elements, 'FCC_A1', T=temp, output='GM')
gamma_x_Ni = np.ravel(gamma_data.X.sel(component='NI'))
gamma_x_Al = np.ravel(gamma_data.X.sel(component='MO'))
gamma_x_Nb = np.ravel(gamma_data.X.sel(component='NB'))
gamma_z = np.ravel(gamma_data.GM)

gamma_interp = SmoothBivariateSpline(gamma_x_Al, gamma_x_Nb, gamma_z)

In [ ]:
def gamma_f(p):
    return float(gamma_interp(p[0], p[1]))

figure, tax = ternary.figure(scale=simplex_res)

tax.heatmapf(gamma_f, boundary=True, style="triangular")
tax.boundary(linewidth=2.0)
tax.set_title(r'$\mathcal{G}_{\gamma}$',fontsize=20)

tax.show()

## Visualize free energy of μ

In [ ]:
mu_data = calculate(tdb, elements, 'MU_phase', T=temp, output='GM')
mu_x_Ni = np.ravel(mu_data.X.sel(component='NI'))
mu_x_Al = np.ravel(mu_data.X.sel(component='MO'))
mu_x_Nb = np.ravel(mu_data.X.sel(component='NB'))
mu_z = np.ravel(mu_data.GM)

mu_interp = SmoothBivariateSpline(mu_x_Al, mu_x_Nb, mu_z)

In [ ]:
def mu_f(p):
    return float(mu_interp(p[0], p[1]))

figure, tax = ternary.figure(scale=simplex_res)

tax.heatmapf(mu_f, boundary=True, style="triangular")
tax.boundary(linewidth=2.0)
tax.set_title(r'$\mathcal{G}_{\gamma\prime}$',fontsize=20)

tax.show()

## Visualize free energy of 𝛿

In [ ]:
delta_data = calculate(tdb, elements, 'delta', T=temp, output='GM')
delta_x_Ni = np.ravel(delta_data.X.sel(component='NI'))
delta_x_Al = np.ravel(delta_data.X.sel(component='MO'))
delta_x_Nb = np.ravel(delta_data.X.sel(component='NB'))
delta_z = np.ravel(delta_data.GM)

delta_interp = SmoothBivariateSpline(delta_x_Al, delta_x_Nb, delta_z)

In [ ]:
def delta_f(p):
    return float(delta_interp(p[0], p[1]))

figure, tax = ternary.figure(scale=simplex_res)

tax.heatmapf(delta_f, boundary=True, style="triangular")
tax.boundary(linewidth=2.0)
tax.set_title(r'$\mathcal{G}_{\gamma\prime\prime}$',fontsize=20)

tax.show()